<a href="https://colab.research.google.com/github/sprksh-crypto/MachineLearningProjects/blob/Classification/buildAndTrainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount drive again.

from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

import tensorflow as tf
import numpy as np

#Bringing over the feature descriptions and parse_example protocol from 3.2.1.
feature_description= {"tickers": tf.io.FixedLenFeature([188], tf.float32, default_value=np.zeros(188)),
                      "weekday": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "hour": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "month": tf.io.FixedLenFeature([], tf.int64, default_value=0),
                      "target":tf.io.FixedLenFeature([], tf.int64, default_value=0)}

def parse_example(serialised):
  examples_parsed=tf.io.parse_example(serialised, feature_description)
  target=examples_parsed.pop('target')
  return examples_parsed, target

#and the dataset that we put through parse_example.
filepath=['/content/drive/My Drive/dataset.tfrecord']

dataset=tf.data.TFRecordDataset(filepath)
datLen=dataset.reduce(0,lambda x,y: x+1)

n_valid=int(datLen.numpy()*.1)
n_test=int(datLen.numpy()*.2)
n_train=datLen.numpy()-n_valid-n_test

dataset_train=dataset.take(n_train).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()
dataset_test=dataset.skip(n_train).take(n_test).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()
dataset_validation=dataset.skip(n_train+n_test).take(n_valid).batch(4096).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 appml-assignment1-dataset-v2.pkl   'Essay Drafts'
'Assignment 1 ML.ipynb'		    'ML textbook code 1.ipynb'
 buildAndTrainModel		     mySavedModel
 buildAndTrainModel.py		    'Physics Project'
 correct.tfrecord		    'Physics Scripts'
 createSavedDataset		     Pics
 customImputeLayerDefinition.ipynb   testhw1
 dataset.tfrecord		     Untitled0.ipynb


In [ ]:
#3.3.1 asks to feed the categorical attribute inputs (weekday,hour,month) into Embedding layers of a dimension of your selection.

#doesn't need to be done for weekday and hour. Only month, because we have 1-12 with an empty 0.

nTokens=12 # inspected the number of values of months.

catEncoder=tf.keras.layers.IntegerLookup(max_tokens=nTokens,num_oov_indices=0)
catEncoder.adapt(dataset_train.map(lambda x,y:x['month']))

#Print to see if working.
#for el in manipulated_dataset.map(lambda x, y:x['month']).take(1):
#  print(el)
#  print(catEncoder(el))

#Bring over keras input dictionary from 3.2.4.

keras_inputs_Dict={'tickers': tf.keras.Input((188), dtype=tf.float32),
                   'weekday': tf.keras.Input((),dtype=tf.int64),
                   'hour': tf.keras.Input((),dtype=tf.int64),
                   'month': tf.keras.Input((),dtype=tf.int64)}

catInts=catEncoder(keras_inputs_Dict['month'])
embDim=32
catEmbs=tf.keras.layers.Embedding(nTokens,embDim)(catInts)
catEmbs1=tf.keras.layers.Embedding(24,embDim)(keras_inputs_Dict['hour'])
catEmbs2=tf.keras.layers.Embedding(7,embDim)(keras_inputs_Dict['weekday'])


In [ ]:
#Bringing over the myImputer class, and where the myImp instance was created.
import tensorflow as tf

class myImputer(tf.keras.layers.experimental.preprocessing.PreprocessingLayer):
  def __init__(self, **kwargs):
    super().__init__( **kwargs)
  def build(self,batch_input_shape):
    self.means=self.add_weight(name='means',shape=(batch_input_shape[-1]), initializer="zeros",trainable=False)
    super().build(batch_input_shape)
  def call(self, X):
    return tf.where(tf.math.is_nan(X),self.means,X)
  def adapt(self, dataset):
    self.build(dataset.element_spec.shape)
    sumOfNonNaNs=dataset.map(lambda z: tf.where(tf.math.is_nan(z), tf.zeros_like(z),z)).reduce(tf.zeros_like(self.means), lambda x,y: x+tf.reduce_sum(y,axis=0))
    numberOfNonNaNs=dataset.map(lambda z: tf.where(tf.math.is_nan(z),tf.zeros_like(z),tf.ones_like(z))).reduce(tf.zeros_like(self.means),lambda x,y: x+tf.reduce_sum(y,axis=0))
    self.means.assign(tf.math.divide(sumOfNonNaNs,numberOfNonNaNs))
  def compute_output_shape(self,batch_input_shape):
    return batch_input_shape

myImp=myImputer()
myImp.adapt(dataset_train.map(lambda x,y: x['tickers']))

In [ ]:
#3.3.2 asks to stack the outputs of the embedding layers and the normaliser to form an input to subsequent layers using tf.concat.

#Bringing over the Keras preprocessing normalization from 3.2.6.

#normalisation layer
normalizer=tf.keras.layers.Normalization(axis=-1)
#targets need to be removed, because we don't want adapt to learn the weights for it.
normalizer.adapt(dataset_train.map(lambda x,y:myImp(x['tickers'])))

preproced=tf.concat([normalizer(myImp(keras_inputs_Dict['tickers'])),catEmbs, catEmbs1, catEmbs2],axis=-1)

In [ ]:
#3.3.3 Include subsequent hidden layers as you wish, but the final output layer should have 23 (22) outputs corresponding
#to the probabilities of the 23 (22) bins of the fractional change and utilize softmax activation

#restMod=tf.keras.Sequential([tf.keras.layers.Dense(22, activation= 'softmax')])

#add hidden and dropout layers?
#restMod=tf.keras.Sequential([tf.keras.layers.Dense(300, activation="relu"), tf.keras.layers.Dense(22, activation= 'softmax')])
restMod=tf.keras.Sequential([tf.keras.layers.Dense(100, activation="sigmoid"), tf.keras.layers.Dense(100, activation="tanh"), tf.keras.layers.Dense(22, activation= 'softmax')])
#restMod=tf.keras.Sequential([tf.keras.layers.Dense(300, activation="relu"),tf.keras.layers.Dropout(rate=0.3), tf.keras.layers.Dense(22, activation= 'softmax')])

In [ ]:
#The keras model is then defined to stretch all the way from the inputs to the 23 outputs.

In [ ]:
#3.3.5 Compile and fit the model in a manner of your choice.

#myOpt=tf.keras.optimizers.SGD(learning_rate=1)
#keras tuner
#keras hyperparameters optimisation

decs=restMod(preproced)
whole_model=tf.keras.Model(inputs=keras_inputs_Dict,outputs=decs)
whole_model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
whole_model.fit(dataset_train, epochs=110, validation_data= dataset_validation, callbacks=[tf.keras.callbacks.EarlyStopping(patience=40,restore_best_weights=True)])
whole_model.evaluate(dataset_test)


Epoch 1/110
7/7 [==============================] - 1s 79ms/step - loss: 2.2939 - accuracy: 0.2066 - val_loss: 2.2984 - val_accuracy: 0.2081
Epoch 2/110
7/7 [==============================] - 0s 44ms/step - loss: 2.2938 - accuracy: 0.2066 - val_loss: 2.2983 - val_accuracy: 0.2081
Epoch 3/110
7/7 [==============================] - 0s 43ms/step - loss: 2.2938 - accuracy: 0.2066 - val_loss: 2.2983 - val_accuracy: 0.2084
Epoch 4/110
7/7 [==============================] - 0s 42ms/step - loss: 2.2937 - accuracy: 0.2067 - val_loss: 2.2982 - val_accuracy: 0.2089
Epoch 5/110
7/7 [==============================] - 0s 45ms/step - loss: 2.2936 - accuracy: 0.2066 - val_loss: 2.2981 - val_accuracy: 0.2086
Epoch 6/110
7/7 [==============================] - 0s 41ms/step - loss: 2.2935 - accuracy: 0.2064 - val_loss: 2.2981 - val_accuracy: 0.2089
Epoch 7/110
7/7 [==============================] - 0s 49ms/step - loss: 2.2935 - accuracy: 0.2064 - val_loss: 2.2980 - val_accuracy: 0.2091
Epoch 8/110
7/7 [===

In [ ]:
#Extras: Figuring out the best batch size.
#import time

#secsReqd=[]

#for batchSize in [32,64,128,512,1024,2048,4096,8*1024]:
#  dataset=tf.data.TFRecordDataset(filepath).batch(batchSize).map(parse_example,num_parallel_calls=tf.data.AUTOTUNE).cache()
#  start_time=time.time()
#  whole_model.fit(dataset,epochs=5,verbose=0)
#  secsReqd.append((time.time()-start_time))
#  print(secsReqd)





In [ ]:
#whole_model.save('/content/drive/My Drive/mySavedModel')

#tf.saved_model.save(whole_model, '/content/drive/My Drive/mySavedModel')

In [ ]:
#My notes:

#Starting with batching at 64, without any hidden layers and without a dropout layer, I got an accuracy of 25%.
#   First mess with batching:
#       @ 32 >>>> too slow, doesn't hit 25% either, around 24%
#       @ 64 >>>> sweet spot.
#       @ 128 >>> worse than 32, 23%

#           If I run this bare bones code, and try to run the "batch optimiser" code, it recommends the larger numbers: 4096, 8*1024, 1024, and 512. In that order.

#       @ 4096 >> this is abismal, accuracy of 15%
#                   >> if then I tweak this, increase # of epochs from 30 to 100, this is consistently at 25%
